# Tutorial de classificação de imagens TensorFlow 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

import pickle
import shutil

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
print(data_dir)

In [ ]:
print(list(data_dir.glob("*")))

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Criar funções para abrir, exibir, manipular e salvar imagens

In [ ]:
import cv2
import numpy as np
import random
from PIL import Image
import os

# Flip Horizontal

In [ ]:
def flip_horizontal(image_path, output_path):
    """ Aplica o flip horizontal em uma imagem e salva no caminho especificado. """
    image_path = os.path.expanduser(image_path)  # Suporte para '~'
    output_path = os.path.expanduser(output_path)
    
    # Carregar a imagem
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ ERRO: Não foi possível carregar a imagem {image_path}")
        return
    
    # Criar pasta de saída, se não existir
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Aplicar flip horizontal
    flipped = cv2.flip(image, 1)
    
    # Salvar a imagem processada
    success = cv2.imwrite(output_path, flipped)
    if not success:
        print(f"❌ ERRO: Não foi possível salvar a imagem em {output_path}")
    else:
        print(f"✅ Flip horizontal salvo em: {output_path}")

def apply_flip_to_dataset(input_dir, output_dir):
    """ Percorre todas as imagens do dataset e aplica o flip horizontal. """
    input_dir = os.path.expanduser(input_dir)  # Caminho absoluto
    output_dir = os.path.expanduser(output_dir)  # Caminho absoluto
    
    # Verificar se o diretório de entrada existe
    if not os.path.exists(input_dir):
        print(f"❌ ERRO: O diretório {input_dir} não existe!")
        return

    # Percorrer todas as classes do dataset
    for class_folder in os.listdir(input_dir):
        class_input_path = os.path.join(input_dir, class_folder)

        # Ignorar arquivos, só processar diretórios (classes)
        if not os.path.isdir(class_input_path):
            continue

        # Criar diretório correspondente na saída
        class_output_path = os.path.join(output_dir, class_folder)
        os.makedirs(class_output_path, exist_ok=True)

        # Percorrer todas as imagens da classe
        for image_file in os.listdir(class_input_path):
            image_input_path = os.path.join(class_input_path, image_file)
            image_output_path = os.path.join(class_output_path, f"{os.path.splitext(image_file)[0]}_flip.jpg")

            # Aplicar flip horizontal
            flip_horizontal(image_input_path, image_output_path)

In [ ]:
# Aplicar flip horizontal
apply_flip_to_dataset("~/.keras/datasets/flower_photos", "augmented_flower_photos_flip")

# Rotação

In [ ]:
def rotate_image(image_path, output_dir, max_angle):
    """ 
    Rotaciona a imagem por um ângulo aleatório entre 0 e max_angle e salva no diretório de saída.
    Preenche pixels indefinidos com preto (RGB 0,0,0).
    """
    image_path = os.path.expanduser(image_path)  # Suporte para '~'
    output_dir = os.path.expanduser(output_dir)  # Caminho absoluto para saída

    # Carregar imagem
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ ERRO: Falha ao carregar {image_path}")
        return
    
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    angle = random.uniform(0, max_angle)  # Escolher ângulo aleatório entre 0 e max_angle

    # Criar matriz de rotação e aplicar a transformação
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, rotation_matrix, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # Criar caminho de saída mantendo a estrutura do dataset original
    class_name = os.path.basename(os.path.dirname(image_path))  # Obtém a classe (ex: 'daisy')
    image_name = os.path.basename(image_path)  # Obtém o nome do arquivo original
    output_class_dir = os.path.join(output_dir, class_name)  # Criar diretório por classe
    os.makedirs(output_class_dir, exist_ok=True)  # Garantir que a pasta existe

    # Criar nome do arquivo de saída
    output_path = os.path.join(output_class_dir, f"{os.path.splitext(image_name)[0]}_rot{int(angle)}.jpg")

    # Salvar a imagem rotacionada
    success = cv2.imwrite(output_path, rotated)

    print(f"{'✅ Rotação aplicada e salva em ' + output_path if success else '❌ Erro ao salvar a imagem!'}")

def apply_rotation_to_dataset(input_dir, base_output_dir, max_angles=[1, 5, 10, 15, 25, 45, 90]):
    """ 
    Gera 7 datasets diferentes, aplicando rotação aleatória entre 0 e X graus. 
    """
    input_dir = os.path.expanduser(input_dir)  # Caminho absoluto
    base_output_dir = os.path.expanduser(base_output_dir)  # Caminho absoluto

    # Verificar se o diretório de entrada existe
    if not os.path.exists(input_dir):
        print(f"❌ ERRO: O diretório {input_dir} não existe!")
        return

    # Criar um dataset separado para cada intervalo de rotação
    for max_angle in max_angles:
        output_dir = f"{base_output_dir}_rot0_{max_angle}"
        print(f"\n📌 Gerando dataset com rotação entre 0 e {max_angle} graus em: {output_dir}")
        
        # Percorrer todas as classes do dataset
        for class_folder in os.listdir(input_dir):
            class_input_path = os.path.join(input_dir, class_folder)

            # Ignorar arquivos, só processar diretórios (classes)
            if not os.path.isdir(class_input_path):
                continue

            # Criar diretório correspondente na saída
            class_output_path = os.path.join(output_dir, class_folder)
            os.makedirs(class_output_path, exist_ok=True)

            # Percorrer todas as imagens da classe
            for image_file in os.listdir(class_input_path):
                image_input_path = os.path.join(class_input_path, image_file)
                rotate_image(image_input_path, output_dir, max_angle)

In [ ]:
# Aplicar rotação e gerar os 7 datasets
apply_rotation_to_dataset("~/.keras/datasets/flower_photos", "augmented_flower_photos_rotated")

# Zoom

In [ ]:
def zoom_image(image_path, output_dir, max_zoom_factor):
    """ 
    Aplica zoom aleatório entre 1.0 e max_zoom_factor e corta para manter as dimensões originais.
    Salva no diretório de saída mantendo a estrutura do dataset original.
    """
    image_path = os.path.expanduser(image_path)  # Suporte para '~'
    output_dir = os.path.expanduser(output_dir)  # Caminho absoluto para saída

    # Carregar imagem
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ ERRO: Falha ao carregar {image_path}")
        return

    h, w, _ = image.shape
    zoom_factor = random.uniform(1.0, max_zoom_factor)  # Escolher um fator de zoom aleatório entre 1 e max_zoom

    # Calcular novas dimensões de corte
    new_h, new_w = int(h / zoom_factor), int(w / zoom_factor)
    start_h = (h - new_h) // 2
    start_w = (w - new_w) // 2

    # Recortar e redimensionar para manter as dimensões originais
    cropped = image[start_h:start_h + new_h, start_w:start_w + new_w]
    zoomed = cv2.resize(cropped, (w, h))

    # Criar caminho de saída mantendo a estrutura do dataset original
    class_name = os.path.basename(os.path.dirname(image_path))  # Obtém a classe (ex: 'daisy')
    image_name = os.path.basename(image_path)  # Obtém o nome do arquivo original
    output_class_dir = os.path.join(output_dir, class_name)  # Criar diretório por classe
    os.makedirs(output_class_dir, exist_ok=True)  # Garantir que a pasta existe

    # Criar nome do arquivo de saída
    output_path = os.path.join(output_class_dir, f"{os.path.splitext(image_name)[0]}_zoom{int((zoom_factor - 1) * 100)}.jpg")

    # Salvar a imagem com zoom
    success = cv2.imwrite(output_path, zoomed)

    print(f"{'✅ Zoom aplicado e salvo em ' + output_path if success else '❌ Erro ao salvar a imagem!'}")

def apply_zoom_to_dataset(input_dir, base_output_dir, max_zoom_factors=[1.05, 1.10, 1.20, 1.40, 1.80]):
    """ 
    Gera 5 datasets diferentes, aplicando zoom aleatório entre 1.0 e Y% de zoom.
    """
    input_dir = os.path.expanduser(input_dir)  # Caminho absoluto
    base_output_dir = os.path.expanduser(base_output_dir)  # Caminho absoluto

    # Verificar se o diretório de entrada existe
    if not os.path.exists(input_dir):
        print(f"❌ ERRO: O diretório {input_dir} não existe!")
        return

    # Criar um dataset separado para cada intervalo de zoom
    for max_zoom in max_zoom_factors:
        output_dir = f"{base_output_dir}_zoom0_{int((max_zoom - 1) * 100)}"
        print(f"\n📌 Gerando dataset com zoom entre 0% e {int((max_zoom - 1) * 100)}% em: {output_dir}")

        # Percorrer todas as classes do dataset
        for class_folder in os.listdir(input_dir):
            class_input_path = os.path.join(input_dir, class_folder)

            # Ignorar arquivos, só processar diretórios (classes)
            if not os.path.isdir(class_input_path):
                continue

            # Criar diretório correspondente na saída
            class_output_path = os.path.join(output_dir, class_folder)
            os.makedirs(class_output_path, exist_ok=True)

            # Percorrer todas as imagens da classe
            for image_file in os.listdir(class_input_path):
                image_input_path = os.path.join(class_input_path, image_file)
                zoom_image(image_input_path, output_dir, max_zoom)

In [ ]:
# Aplicar zoom e gerar os 5 datasets
apply_zoom_to_dataset("~/.keras/datasets/flower_photos", "augmented_flower_photos_zoom")

# Incluir o Dataset original nos Datasets aumentados

In [ ]:
def copy_original_to_augmented(original_dir, augmented_dirs):
    """
    Copia todas as imagens do dataset original para os datasets aumentados,
    garantindo que as classes originais estejam presentes nos aumentos.
    
    original_dir: Caminho do dataset original (ex: ~/.keras/datasets/flower_photos)
    augmented_dirs: Lista de diretórios dos datasets aumentados
    """
    original_dir = os.path.expanduser(original_dir)  # Suporte para '~'
    augmented_dirs = [os.path.expanduser(d) for d in augmented_dirs]  # Expande caminhos

    # Verifica se o diretório original existe
    if not os.path.exists(original_dir):
        print(f"ERRO: O diretório original '{original_dir}' não existe!")
        return
    
    # Iterar sobre cada dataset aumentado
    for augmented_dir in augmented_dirs:
        print(f"\nCopiando imagens do dataset original para: {augmented_dir}")

        # Criar diretórios das classes no dataset aumentado
        for class_folder in os.listdir(original_dir):
            class_path_original = os.path.join(original_dir, class_folder)
            class_path_augmented = os.path.join(augmented_dir, class_folder)

            # Verifica se é um diretório (ignora arquivos)
            if not os.path.isdir(class_path_original):
                continue

            # Cria a pasta no dataset aumentado se não existir
            os.makedirs(class_path_augmented, exist_ok=True)

            # Copia todas as imagens da classe original para a classe do dataset aumentado
            for image_file in os.listdir(class_path_original):
                src = os.path.join(class_path_original, image_file)
                dst = os.path.join(class_path_augmented, image_file)

                # Copiar somente se ainda não estiver no dataset aumentado
                if not os.path.exists(dst):
                    shutil.copy2(src, dst)

        print(f"Cópia concluída para {augmented_dir}")

In [ ]:
# Lista dos diretórios aumentados
augmented_datasets = [
    "augmented_flower_photos_flip",
    "augmented_flower_photos_rotated_rot0_1",
    "augmented_flower_photos_rotated_rot0_5",
    "augmented_flower_photos_rotated_rot0_10",
    "augmented_flower_photos_rotated_rot0_15",
    "augmented_flower_photos_rotated_rot0_25",
    "augmented_flower_photos_rotated_rot0_45",
    "augmented_flower_photos_rotated_rot0_90",
    "augmented_flower_photos_zoom_zoom0_5",
    "augmented_flower_photos_zoom_zoom0_10",
    "augmented_flower_photos_zoom_zoom0_20",
    "augmented_flower_photos_zoom_zoom0_40",
    "augmented_flower_photos_zoom_zoom0_80"
]

In [ ]:
# Copiar o dataset original para os datasets aumentados
copy_original_to_augmented("~/.keras/datasets/flower_photos", augmented_datasets)

# Contagem dos dados dos Datasets

In [ ]:
def count_images_in_dataset(dataset_path, dataset_name):
    """ Conta a quantidade total de imagens no dataset e exibe os resultados. """
    dataset_path = os.path.expanduser(dataset_path)  # Garante caminho absoluto
    total_images = 0
    class_counts = {}  # Contador por classe

    # Percorre todas as pastas dentro do dataset
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)

        # Ignorar arquivos, só processar diretórios (classes)
        if not os.path.isdir(class_path):
            continue

        # Conta as imagens dentro da classe
        num_images = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        total_images += num_images
        class_counts[class_folder] = num_images

    # Exibir resultados
    print(f"\n**Resumo do dataset: {dataset_name}**")
    print(f"   Total de imagens: {total_images}")
    for class_name, count in class_counts.items():
        print(f"   Classe '{class_name}': {count} imagens")

    return total_images, class_counts

# Contar imagens no dataset original
original_total, original_counts = count_images_in_dataset("~/.keras/datasets/flower_photos", "Original")

# Contar imagens no dataset aumentado
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_flip", "Flip")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_1", "Rotação x a 1º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_5", "Rotação x a 5º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_10", "Rotação x a 10º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_15", "Rotação x a 15º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_25", "Rotação x a 25º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_45", "Rotação x a 45º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_rotated_rot0_90", "Rotação x a 90º")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_zoom_zoom0_5", "Zoom y a 5%")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_zoom_zoom0_10", "Zoom y a 10%")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_zoom_zoom0_19", "Zoom y a 20%")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_zoom_zoom0_39", "Zoom y a 40%")
augmented_total, augmented_counts = count_images_in_dataset("augmented_flower_photos_zoom_zoom0_80", "Zoom y a 80%")

# TREINAMENTO DE TODOS OS DATASETS (FLIP, ROTAÇÃO E ZOOM)

In [2]:
batch_size = 32
img_height = 180
img_width = 180
epochs = 10

In [3]:
# Lista de datasets para Flip, Rotação e Zoom
datasets = [
    "augmented_flower_photos_flip",
    "augmented_flower_photos_rotated_rot0_1",
    "augmented_flower_photos_rotated_rot0_5",
    "augmented_flower_photos_rotated_rot0_10",
    "augmented_flower_photos_rotated_rot0_15",
    "augmented_flower_photos_rotated_rot0_25",
    "augmented_flower_photos_rotated_rot0_45",
    "augmented_flower_photos_rotated_rot0_90",
    "augmented_flower_photos_zoom_zoom0_5",
    "augmented_flower_photos_zoom_zoom0_10",
    "augmented_flower_photos_zoom_zoom0_20",
    "augmented_flower_photos_zoom_zoom0_40",
    "augmented_flower_photos_zoom_zoom0_80"
]

In [4]:
# Criar um dicionário para armazenar os históricos de treinamento
training_histories = {}

In [5]:
for dataset in datasets:
    print(f"\n Treinando modelo para dataset: {dataset}")

    # Carregar o dataset
    train_ds = tf.keras.utils.image_dataset_from_directory(
        dataset,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        dataset,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    class_names = train_ds.class_names
    num_classes = len(class_names)

    # Criar o modelo para treinamento
    model = keras.Sequential([
        layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.summary()

    # Treinar o modelo
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    # Salvar o histórico de treinamento
    training_histories[dataset] = history.history

    # Salvar o modelo treinado
    model.save(f"model_{dataset}.h5")

    # Salvar o histórico de treinamento em um arquivo
    with open(f"history_{dataset}.pkl", "wb") as f:
        pickle.dump(history.history, f)

    print(f" Modelo treinado e salvo para {dataset}!")

print("\n Todos os modelos foram treinados e salvos com sucesso!")


 Treinando modelo para dataset: augmented_flower_photos_flip
Found 7340 files belonging to 5 classes.
Using 5872 files for training.


2025-02-04 22:22:13.849172: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-02-04 22:22:13.849205: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-02-04 22:22:13.849214: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-02-04 22:22:13.849781: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-04 22:22:13.850039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 7340 files belonging to 5 classes.
Using 1468 files for validation.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 90, 90, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 45, 45, 32)        0         
 g2D)                                                            
                                                

2025-02-04 22:22:14.632766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


184/184 [==============================] - ETA: 0s - loss: 1.2299 - accuracy: 0.5043

...

184/184 [==============================] - 13s 67ms/step - loss: 1.2518 - accuracy: 0.4719 - val_loss: 1.1701 - val_accuracy: 0.5341
Epoch 2/10
184/184 [==============================] - 12s 66ms/step - loss: 1.3261 - accuracy: 0.5296 - val_loss: 1.1457 - val_accuracy: 0.5429
Epoch 3/10
184/184 [==============================] - 12s 65ms/step - loss: 1.0718 - accuracy: 0.6034 - val_loss: 1.1906 - val_accuracy: 0.5422
Epoch 4/10
184/184 [==============================] - 12s 66ms/step - loss: 1.2228 - accuracy: 0.5644 - val_loss: 1.2007 - val_accuracy: 0.5770
Epoch 5/10
184/184 [==============================] - 12s 66ms/step - loss: 1.2762 - accuracy: 0.5651 - val_loss: 0.9814 - val_accuracy: 0.6015
Epoch 6/10
184/184 [==============================] - 12s 66ms/step - loss: 0.9323 - accuracy: 0.6628 - val_loss: 1.0626 - val_accuracy: 0.5845
Epoch 7/10
184/184 [==============================] - 12s 65ms/step - loss: 0.7636 - accuracy: 0.7200 - val_loss: 1.0005 - val_accuracy: 0.6458
Epo

2025-02-04 22:44:57.696648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


184/184 [==============================] - ETA: 0s - loss: 1.2875 - accuracy: 0.4825

2025-02-04 22:45:09.198303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


184/184 [==============================] - 13s 66ms/step - loss: 1.2875 - accuracy: 0.4825 - val_loss: 1.2527 - val_accuracy: 0.5061
Epoch 2/10
184/184 [==============================] - 12s 66ms/step - loss: 1.5886 - accuracy: 0.5330 - val_loss: 4.6468 - val_accuracy: 0.4659
Epoch 3/10
184/184 [==============================] - 12s 66ms/step - loss: 5.9195 - accuracy: 0.4865 - val_loss: 10.7589 - val_accuracy: 0.4734
Epoch 4/10
184/184 [==============================] - 12s 67ms/step - loss: 62.5818 - accuracy: 0.4431 - val_loss: 121.0265 - val_accuracy: 0.4087
Epoch 5/10
184/184 [==============================] - 13s 68ms/step - loss: 452.6786 - accuracy: 0.4303 - val_loss: 1009.1532 - val_accuracy: 0.3665
Epoch 6/10
184/184 [==============================] - 12s 66ms/step - loss: 2145.4253 - accuracy: 0.3939 - val_loss: 2963.7163 - val_accuracy: 0.4278
Epoch 7/10
184/184 [==============================] - 12s 67ms/step - loss: 6052.9419 - accuracy: 0.3770 - val_loss: 16555.9629 - va

# Acurácia final de cada modelo

In [6]:
for dataset in datasets:
    history_file = f"history_{dataset}.pkl"

    # Verificar se o arquivo de histórico existe
    if os.path.exists(history_file):
        with open(history_file, "rb") as f:
            history = pickle.load(f)
        
        # Pegando a última acurácia no conjunto de validação
        final_accuracy = history['val_accuracy'][-1] * 100  # Converte para porcentagem
        print(f"{dataset}: {final_accuracy:.2f}%")
    else:
        print(f"Histórico não encontrado para {dataset}!")

augmented_flower_photos_flip: 59.40%
augmented_flower_photos_rotated_rot0_1: 62.19%
augmented_flower_photos_rotated_rot0_5: 67.03%
augmented_flower_photos_rotated_rot0_10: 46.46%
augmented_flower_photos_rotated_rot0_15: 63.08%
augmented_flower_photos_rotated_rot0_25: 66.96%
augmented_flower_photos_rotated_rot0_45: 47.00%
augmented_flower_photos_rotated_rot0_90: 43.94%
augmented_flower_photos_zoom_zoom0_5: 52.32%
augmented_flower_photos_zoom_zoom0_10: 64.44%
augmented_flower_photos_zoom_zoom0_20: 44.82%
augmented_flower_photos_zoom_zoom0_40: 64.92%
augmented_flower_photos_zoom_zoom0_80: 34.26%
